In [1]:
# --- Block 5: Walk-Forward Cross-Validation + Hyperparameter-Search ---
import os, sys, json, time, logging, glob, re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ROOT = os.path.abspath("..")
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

# TensorFlow logging ruhigstellen
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

In [2]:
# ---- Core-Config laden -------------------------------------------------
with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)

TICKER, START, END, INTERVAL = C["ticker"], C["start"], C["end"], C["interval"]
HORIZON  = int(C["horizon"])
LOOKBACK_DEFAULT = int(C["lookback"])
BATCH    = int(C.get("batch", 64))
SEED     = int(C.get("seed", 42))
FEATURESET = C.get("featureset", "v2")
EPS_MODE   = C.get("epsilon_mode", "abs")
EPSILON    = float(C.get("epsilon", 0.0005))
RESULTS_DIR = Path(C.get("results_dir", "../results"))

np.random.seed(SEED); tf.random.set_seed(SEED)

# WFCV Run-Ordner
RUN_DIR = RESULTS_DIR / time.strftime("%Y-%m-%d_%H-%M-%S_wfcv")
(RUN_DIR / "plots").mkdir(parents=True, exist_ok=True)
print("WFCV_RUN_DIR:", RUN_DIR)

WFCV_RUN_DIR: ..\results\2025-10-22_12-13-21_wfcv


In [3]:
# ---- Optional: schneller Smoke-Test -----------------------------------
FAST = C.get("fast_wfcv", False)  # CHANGE: auch aus config.json aktivierbar
EPOCHS_GRID = 60
N_FOLDS = 5
if FAST:
    EPOCHS_GRID = 25
    N_FOLDS = 3

In [4]:
# ---- Daten & Features --------------------------------------------------
import yaml

yaml_path = f"../data/features_{FEATURESET}.yml"
meta = {}
label_h = label_mode = label_eps = None

if os.path.exists(yaml_path):
    with open(yaml_path, "r") as f:
        meta = yaml.safe_load(f) or {}
    lab = (meta or {}).get("label", {})
    label_h    = lab.get("horizon", None)
    label_mode = lab.get("mode", None)
    label_eps  = lab.get("epsilon", None)

def _parse_h_meps_from_name(path: str):
    mH = re.search(r"_cls_h(\d+)_", path)
    me = re.search(r"_(abs|rel)(\d+p\d+)\.csv$", path)
    H  = int(mH.group(1)) if mH else None
    md = me.group(1) if me else None
    eps= float(me.group(2).replace("p",".")) if me else None
    return H, md, eps

def _infer_from_existing_files(tkr, itv, start, end, mode_hint=None, eps_hint=None):
    pat = f"../data/{tkr}_{itv}_{start}_{end}_cls_h*_.csv".replace("_ .csv",".csv")
    cands = sorted(glob.glob(pat), key=os.path.getmtime)
    if mode_hint and (eps_hint is not None):
        tag = f"{mode_hint}{str(eps_hint).replace('.','p')}"
        cands = [c for c in cands if c.endswith(f"_{tag}.csv")]
    if not cands:
        return None
    return _parse_h_meps_from_name(cands[-1])

H_FOR_FILE    = int(label_h)    if label_h    is not None else None
MODE_FOR_FILE = str(label_mode) if label_mode is not None else None
EPS_FOR_FILE  = float(label_eps) if label_eps is not None else None

if (H_FOR_FILE is None) or (MODE_FOR_FILE is None) or (EPS_FOR_FILE is None):
    inferred = _infer_from_existing_files(TICKER, INTERVAL, START, END,
                                          mode_hint=MODE_FOR_FILE, eps_hint=EPS_FOR_FILE)
    if inferred is not None:
        H_i, M_i, E_i = inferred
        H_FOR_FILE    = H_FOR_FILE    if H_FOR_FILE    is not None else H_i
        MODE_FOR_FILE = MODE_FOR_FILE if MODE_FOR_FILE is not None else M_i
        EPS_FOR_FILE  = EPS_FOR_FILE  if EPS_FOR_FILE  is not None else E_i

if (H_FOR_FILE is None) or (MODE_FOR_FILE is None) or (EPS_FOR_FILE is None):
    raise RuntimeError(
        "Label-Definition (H/mode/epsilon) konnte nicht aus YAML oder existierenden CSV-Dateien bestimmt werden.\n"
        f"Erwartet YAML unter: {yaml_path}\n"
        "Oder eine Datei wie: ../data/"
        f"{TICKER}_{INTERVAL}_{START}_{END}_cls_h<H>_<abs|rel><epsilon_mit_p>.csv"
    )

eps_tag   = f"{MODE_FOR_FILE}{str(EPS_FOR_FILE).replace('.','p')}"
TRAIN_CSV = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{H_FOR_FILE}_{eps_tag}.csv"

if not os.path.exists(TRAIN_CSV):
    pat = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h*_{eps_tag}.csv"
    candidates = sorted(glob.glob(pat), key=os.path.getmtime)
    if candidates:
        TRAIN_CSV = candidates[-1]
    else:
        raise FileNotFoundError(
            f"Train CSV nicht gefunden: {TRAIN_CSV}\n"
            f"Gesucht nach Pattern: {pat}\n"
            "Hinweis: Block 2 mit dieser Label-Definition laufen lassen."
        )

print("Loaded TRAIN_CSV:", TRAIN_CSV)
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=True).sort_index()

OHLCV = {"open","high","low","close","volume"}
if meta:
    FEATURES_ALL = [c for c in meta.get("features", []) if c in df.columns]
else:
    FEATURES_ALL = [c for c in df.columns if c not in (OHLCV | {"target"})]
assert len(FEATURES_ALL) > 0, "Keine Features gefunden."

X_full = df[FEATURES_ALL].copy()
y_full = df["target"].astype(int).copy()

print("Label pos_rate:", round(y_full.mean(), 3), "| n:", len(y_full))

Loaded TRAIN_CSV: ../data/AAPL_1d_2012-01-01_2025-09-01_cls_h1_abs0p0005.csv
Label pos_rate: 0.509 | n: 3402


In [5]:
# ---- Walk-Forward Splits (fix: 5 Folds, val=20%, min_train=45%) -------
def make_wf_splits(n, n_folds=5, val_frac=0.20, min_train_frac=0.45):
    val_len   = max(60, int(round(n * val_frac)))
    min_train = max(200, int(round(n * min_train_frac)))
    start_val_end = min_train + val_len
    if start_val_end + 1 > n:
        raise ValueError(f"Dataset zu kurz für val_frac/min_train_frac: n={n}, "
                         f"min_train={min_train}, val_len={val_len}")
    val_ends = np.linspace(start_val_end, n, num=n_folds, endpoint=True).astype(int)
    val_ends = np.unique(val_ends)
    if len(val_ends) < n_folds:
        step = max(1, (n - start_val_end) // n_folds)
        val_ends = np.arange(start_val_end, start_val_end + step * n_folds, step)
        val_ends = np.clip(val_ends, start_val_end, n)
    stops = []
    for ve in val_ends[:n_folds]:
        te = int(ve - val_len)
        te = max(te, LOOKBACK_DEFAULT + 1)
        if te <= 0 or ve <= te or ve > n:
            continue
        stops.append((slice(0, te), slice(te, ve)))
    if len(stops) != n_folds:
        raise RuntimeError(f"Erzeugte nur {len(stops)} von {n_folds} Folds.")
    return stops

n = len(df)
splits = make_wf_splits(n, n_folds=N_FOLDS, val_frac=0.20, min_train_frac=0.45)
print("Anzahl Folds:", len(splits))
if len(splits) != N_FOLDS:
    raise RuntimeError("Es sind nicht exakt die gewünschten Folds entstanden.")

Anzahl Folds: 5


In [6]:
# ---- Hilfsfunktionen: Windowing + Pipeline -----------------------------
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef, average_precision_score, roc_auc_score
from tensorflow.keras import layers, regularizers, callbacks, optimizers, models

def make_windows(X_df, y_ser, lookback):
    Xv = X_df.values.astype(np.float32)
    yv = y_ser.values.astype(np.int32)
    xs, ys = [], []
    for i in range(lookback-1, len(X_df)):
        xs.append(Xv[i - lookback + 1 : i + 1])
        ys.append(yv[i])
    return np.stack(xs, axis=0), np.array(ys)

def build_model(n_features, width1=64, width2=32, dropout=0.10, lr=5e-4, use_gru=True):
    rnn = layers.GRU if use_gru else layers.LSTM
    m = models.Sequential([
        layers.Input(shape=(None, n_features)),
        rnn(width1, return_sequences=True, recurrent_dropout=dropout),
        layers.LayerNormalization(),
        rnn(width2, recurrent_dropout=dropout),
        layers.LayerNormalization(),
        layers.Dense(16, activation="relu", kernel_regularizer=regularizers.l2(1e-5)),
        layers.Dense(1, activation="sigmoid"),
    ])
    m.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc"),
                 tf.keras.metrics.AUC(name="auprc", curve="PR")]
    )
    return m

# CHANGE: MCC @ best threshold (auf Validation) – nicht fix 0.5
def mcc_at_best_thr(y_true, y_prob):
    ts = np.r_[0.0, np.unique(y_prob), 1.0]
    best = (-1.0, 0.5)
    for t in ts:
        m = matthews_corrcoef(y_true, (y_prob >= t).astype(int))
        if m > best[0]:
            best = (float(m), float(t))
    return best  # (mcc, thr)

def fit_eval_fold(X_tr, y_tr, X_va, y_va, lookback, hp, seed=SEED, batch=BATCH, epochs=EPOCHS_GRID):
    tf.keras.backend.clear_session()

    # Scaler nur auf Train
    scaler = StandardScaler()
    X_tr_s = pd.DataFrame(scaler.fit_transform(X_tr), index=X_tr.index, columns=X_tr.columns)
    X_va_s = pd.DataFrame(scaler.transform(X_va),     index=X_va.index, columns=X_va.columns)

    Xtr_win, ytr = make_windows(X_tr_s, y_tr, lookback)
    Xva_win, yva = make_windows(X_va_s, y_va, lookback)

    ds_tr = tf.data.Dataset.from_tensor_slices((Xtr_win, ytr)) \
            .shuffle(len(Xtr_win), seed=seed) \
            .batch(batch).prefetch(tf.data.AUTOTUNE)
    ds_va = tf.data.Dataset.from_tensor_slices((Xva_win, yva)) \
            .batch(batch).prefetch(tf.data.AUTOTUNE)

    tf.keras.utils.set_random_seed(seed)
    model = build_model(
        n_features=Xtr_win.shape[-1],
        width1=hp["width1"], width2=hp["width2"],
        dropout=hp["dropout"], lr=hp["lr"], use_gru=(hp["cell"]=="GRU")
    )

    cbs = [
        callbacks.EarlyStopping(monitor="val_auprc", mode="max", patience=6, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(monitor="val_auprc", mode="max", factor=0.5, patience=3, min_lr=1e-5),
        callbacks.TerminateOnNaN(),
    ]

    print(f"  -> fit (epochs={epochs}, batch={batch}) ...", flush=True)
    hist = model.fit(ds_tr, validation_data=ds_va, epochs=epochs, verbose=1 if FAST else 0, callbacks=cbs)

    yva_proba = model.predict(ds_va, verbose=0).ravel()

    # ... nach dem Val-Predict:
    mcc_val, thr_val = mcc_at_best_thr(yva, yva_proba)   # wie in deiner Suche
    yva_pred_best = (yva_proba >= thr_val).astype(int)

    metrics = dict(
        mcc=float(mcc_val),
        thr_val=float(thr_val),
        bal_acc=float(balanced_accuracy_score(yva, yva_pred_best)),  # <— zurück
        auprc=float(average_precision_score(yva, yva_proba)),
        auroc=float(roc_auc_score(yva, yva_proba)),
        epochs_trained=int(len(hist.history["loss"]))
    )
    tf.keras.backend.clear_session()
    return metrics

In [7]:
# --- Grids --------------------------------------------------------------
# CHANGE: aussagekräftiges, aber kompaktes Grid (per Spezifikation)
LOOKBACK_GRID = [40, 60, 120] if not FAST else [LOOKBACK_DEFAULT]
HP_GRID = [
    dict(width1=w1, width2=w2, dropout=dp, lr=lr, cell=cell)
    for (w1, w2) in [(32,16), (64,32)]
    for lr in [5e-4, 3e-4]
    for dp in [0.0, 0.1, 0.2]
    for cell in ["GRU", "LSTM"]
] if not FAST else [dict(width1=32, width2=16, dropout=0.10, lr=5e-4, cell="GRU")]

# CHANGE: neues Feature-Subset „mom+vol“
FEATURE_SUBSETS = {
    "all": FEATURES_ALL,
    "mom_only": [c for c in FEATURES_ALL
                 if ("logret" in c) or ("macd" in c) or (c in {"sma_diff","rsi_14","bb_pos"})],
    "mom+vol": [c for c in FEATURES_ALL
                if (("logret" in c) or ("macd" in c) or (c in {"sma_diff","rsi_14","bb_pos"}))
                   or (c in {"realized_vol_10","vol_z_20"})]   # <- Ergänzung
}

print("Sanity check vor der Suche:")
print("  n Zeilen df:", len(df))
print("  FEATURESET:", FEATURESET)
print("  FEATURES_ALL:", len(FEATURES_ALL))
print("  FEATURE_SUBSETS:", list(FEATURE_SUBSETS.keys()))
print("  LOOKBACK_GRID:", LOOKBACK_GRID)
print("  HP_GRID:", (len(HP_GRID) if not FAST else "FAST=1"))
print("  #splits:", len(splits))
if len(splits) > 0:
    tr_s, va_s = splits[0]
    print("  Fold0 sizes (train/val):", tr_s.stop, va_s.stop - va_s.start)

Sanity check vor der Suche:
  n Zeilen df: 3402
  FEATURESET: v2
  FEATURES_ALL: 11
  FEATURE_SUBSETS: ['all', 'mom_only', 'mom+vol']
  LOOKBACK_GRID: [40, 60, 120]
  HP_GRID: 24
  #splits: 5
  Fold0 sizes (train/val): 1531 680


In [8]:
# ---- Suche (FAST PATCH) ------------------------------------------------------
from time import perf_counter
import pandas as pd

print("Starte FAST-Suche ...", flush=True)

# ---------- Fast-Overrides nur für diese Zelle ----------
FAST_PATCH     = True
MAX_SECONDS    = 60 * 60        # 60 min Zeitbudget
EPOCHS_FAST    = 25             # statt 60
NFOLDS_FAST    = 3              # statt 5
BATCH_FAST     = 128            # größerer Batch -> weniger Steps/Epoche

# reduziere Subsets/Lookbacks/HPs radikal für <~1h auf CPU
FEATURE_SUBSETS_FAST = {}
# nimm bevorzugt "mom+vol", sonst "mom_only", sonst 1. Eintrag:
if "mom+vol" in FEATURE_SUBSETS:
    FEATURE_SUBSETS_FAST["mom+vol"] = FEATURE_SUBSETS["mom+vol"]
elif "mom_only" in FEATURE_SUBSETS:
    FEATURE_SUBSETS_FAST["mom_only"] = FEATURE_SUBSETS["mom_only"]
else:
    k0 = next(iter(FEATURE_SUBSETS.keys()))
    FEATURE_SUBSETS_FAST[k0] = FEATURE_SUBSETS[k0]

LOOKBACK_GRID_FAST = [60]       # ein Lookback
HP_GRID_FAST = [                # eine solide, schnelle Kombi
    dict(width1=32, width2=16, dropout=0.10, lr=5e-4, cell="GRU"),
]

# Wende Overrides an (nur in dieser Zelle)
if FAST_PATCH:
    EPOCHS_GRID = EPOCHS_FAST
    # begrenze auf die ersten N_FOLDS
    splits = splits[:min(NFOLDS_FAST, len(splits))]
    FEATURE_SUBSETS = FEATURE_SUBSETS_FAST
    LOOKBACK_GRID = LOOKBACK_GRID_FAST
    HP_GRID = HP_GRID_FAST

# ---------- Resume-Unterstützung & Sofort-Checkpoint ----------
csv_path = RUN_DIR / "wfcv_results.csv"
records = []
total_combos = 0
t0 = perf_counter()

# bereits existierende Ergebnisse zum Skipping laden
if csv_path.exists():
    done_df = pd.read_csv(csv_path)                    # <-- IMMER definieren
    done_keys = {
        (r["features_used"], int(r["lookback"]),
         r["cell"], int(r["width1"]), int(r["width2"]),
         float(r["dropout"]), float(r["lr"]), int(r["fold"]))
        for _, r in done_df.iterrows()
    }
else:
    done_df = pd.DataFrame()                           # <-- leerer Frame
    done_keys = set()


def _size(slc):  # helper
    return slc.stop - (slc.start or 0)

stop_time = t0 + MAX_SECONDS
print(f"[FAST] Grid: subsets={list(FEATURE_SUBSETS.keys())}, LB={LOOKBACK_GRID}, HPs={len(HP_GRID)}, folds={len(splits)}")
print(f"[FAST] Budget: {MAX_SECONDS//60:.0f} min, EPOCHS={EPOCHS_GRID}, BATCH={BATCH_FAST}\n")

for feat_name, FEATS in FEATURE_SUBSETS.items():
    if len(FEATS) == 0:
        print(f"[{feat_name}] übersprungen: 0 Features.")
        continue

    for lookback in LOOKBACK_GRID:
        for hp in HP_GRID:
            total_combos += 1
            ran_folds = 0

            for fold_id, (tr_s, va_s) in enumerate(splits, start=1):
                key = (feat_name, int(lookback),
                       hp["cell"], int(hp["width1"]), int(hp["width2"]),
                       float(hp["dropout"]), float(hp["lr"]), int(fold_id))
                if key in done_keys:
                    print(f"[{feat_name} | LB={lookback} | {hp['cell']} {hp['width1']}/{hp['width2']} "
                          f"dp={hp['dropout']} lr={hp['lr']}] Fold{fold_id}: SKIP (bereits vorhanden)")
                    ran_folds += 1
                    continue

                    # Zeitbudget prüfen
                if perf_counter() > stop_time:
                    print("[INFO] Zeitbudget erreicht — speichere & beende.")
                    # vorhandene + neue records sicher schreiben
                    out_df = pd.concat([done_df, pd.DataFrame.from_records(records)], ignore_index=True) if csv_path.exists() else pd.DataFrame.from_records(records)
                    out_df.to_csv(csv_path, index=False)
                    raise SystemExit(0)

                X_tr, y_tr = X_full.iloc[tr_s][FEATS], y_full.iloc[tr_s]
                X_va, y_va = X_full.iloc[va_s][FEATS], y_full.iloc[va_s]

                # Minimalcheck für Windowing
                if len(X_tr) < (lookback + 50) or len(X_va) < (lookback + 10):
                    print(f"[{feat_name} | LB={lookback} | {hp['cell']} {hp['width1']}/{hp['width2']} "
                          f"dp={hp['dropout']} lr={hp['lr']}] Skip-Fold (zu kurz): "
                          f"train={len(X_tr)}, val={len(X_va)}, needed>={(lookback+50)}/{(lookback+10)}")
                    continue

                # NOTE: wir erzwingen hier batch=BATCH_FAST und epochs=EPOCHS_GRID
                mets = fit_eval_fold(X_tr, y_tr, X_va, y_va,
                                     lookback=lookback, hp=hp,
                                     epochs=EPOCHS_GRID, batch=BATCH_FAST)

                ran_folds += 1
                rec = {
                    "feature_set": FEATURESET,
                    "features_used": feat_name,
                    "n_features": len(FEATS),
                    "lookback": lookback,
                    **hp,
                    "fold": fold_id,
                    **mets
                }
                records.append(rec)

                # Sofort persistieren (robust bei Abbruch)
                cur_df = pd.DataFrame.from_records(records)
                out_df = pd.concat([done_df, cur_df], ignore_index=True)
                out_df.to_csv(csv_path, index=False)
                done_df = out_df                                      # <-- in-memory aktualisieren

                # Log – expects mets['mcc'] und mets['thr_val'] (aus mcc_at_best_thr)
                thr_str = f"{mets.get('thr_val', 0.5):.3f}" if 'thr_val' in mets else "n/a"
                print(f"[{feat_name} | LB={lookback} | {hp['cell']} {hp['width1']}/{hp['width2']} "
                      f"dp={hp['dropout']} lr={hp['lr']}] Fold{fold_id}: "
                      f"MCC@best={mets['mcc']:.3f} thr={thr_str} AUPRC={mets['auprc']:.3f}")

            print(f"--> Summary [{feat_name} | LB={lookback} | {hp['cell']} {hp['width1']}/{hp['width2']}] "
                  f"ran_folds={ran_folds}/{len(splits)}")

t1 = perf_counter()
print(f"\nSuche fertig: combos={total_combos}, records={len(records)}, Dauer={t1-t0:.1f}s")

# finaler Merge (falls während der Suche schon geschrieben wurde)
final_df = pd.read_csv(csv_path) if csv_path.exists() else pd.DataFrame.from_records(records)
final_df.to_csv(csv_path, index=False)
print("Geschrieben:", csv_path)

Starte FAST-Suche ...
[FAST] Grid: subsets=['mom+vol'], LB=[60], HPs=1, folds=3
[FAST] Budget: 60 min, EPOCHS=25, BATCH=128

  -> fit (epochs=25, batch=128) ...
[mom+vol | LB=60 | GRU 32/16 dp=0.1 lr=0.0005] Fold1: MCC@best=0.106 thr=0.635 AUPRC=0.562
  -> fit (epochs=25, batch=128) ...
[mom+vol | LB=60 | GRU 32/16 dp=0.1 lr=0.0005] Fold2: MCC@best=0.051 thr=0.628 AUPRC=0.513
  -> fit (epochs=25, batch=128) ...
[mom+vol | LB=60 | GRU 32/16 dp=0.1 lr=0.0005] Fold3: MCC@best=0.041 thr=0.418 AUPRC=0.501
--> Summary [mom+vol | LB=60 | GRU 32/16] ran_folds=3/3

Suche fertig: combos=1, records=3, Dauer=15.2s
Geschrieben: ..\results\2025-10-22_12-13-21_wfcv\wfcv_results.csv


In [9]:
# ---- Aggregation & Best-Config (robust aus CSV) ------------------------------
import pandas as pd, json, numpy as np

csv_path = RUN_DIR / "wfcv_results.csv"
assert csv_path.exists(), f"wfcv_results.csv fehlt unter {csv_path} – zuerst die Suche laufen lassen."

results = pd.read_csv(csv_path)

# Dedupe (Resume-Fälle)
key_cols = ["features_used","lookback","cell","width1","width2","dropout","lr","fold"]
present_keys = [c for c in key_cols if c in results.columns]
results = results.drop_duplicates(subset=present_keys, keep="last").reset_index(drop=True)

# Typen glätten (falls Strings reingerutscht sind)
for col in ["lookback","width1","width2","fold"]:
    if col in results.columns:
        results[col] = pd.to_numeric(results[col], errors="coerce").astype("Int64")
for col in ["dropout","lr"]:
    if col in results.columns:
        results[col] = pd.to_numeric(results[col], errors="coerce")

# Pflichtmetriken
need_cols = {"mcc","auprc","auroc"}
missing = need_cols - set(results.columns)
if missing:
    raise ValueError(f"Fehlende Spalten in wfcv_results.csv: {sorted(missing)}")

# Gruppierschlüssel (nur vorhandene)
agg_cols = [c for c in ["feature_set","features_used","n_features","lookback",
                        "width1","width2","dropout","lr","cell"] if c in results.columns]

# Aggregation; bal_acc optional
agg_dict = {"mcc": ["mean","std"], "auprc": ["mean","std"], "auroc": ["mean"]}
if "bal_acc" in results.columns:
    agg_dict["bal_acc"] = ["mean"]

g = results.groupby(agg_cols).agg(agg_dict)

# MultiIndex-Spalten flatten (robust)
g.columns = [
    "_".join([str(x) for x in col if str(x) != ""]).strip("_")
    for col in g.columns.to_flat_index()
]
g = g.reset_index()

# n_folds sauber mergen (statt .values)
nf = results.groupby(agg_cols).size().reset_index(name="n_folds")
g = g.merge(nf, on=agg_cols, how="left")

# Benennungen hübsch (nur falls nötig)
rename_map = {
    "mcc_mean":"mcc_mean", "mcc_std":"mcc_std",
    "auprc_mean":"auprc_mean", "auprc_std":"auprc_std",
    "auroc_mean":"auroc_mean", "bal_acc_mean":"balacc_mean"
}
g = g.rename(columns=rename_map)

# Ranking
g = g.sort_values(["mcc_mean","auprc_mean","mcc_std"], ascending=[False, False, True])

# Artefakte
(g).to_csv(RUN_DIR / "wfcv_results_agg.csv", index=False)
top5 = g.head(5).copy()
top5.to_csv(RUN_DIR / "wfcv_results_top5.csv", index=False)

best = top5.iloc[0].to_dict() if len(top5) else {}
with open(RUN_DIR / "best_config.json", "w") as f:
    json.dump(best, f, indent=2)

print("Best config:", best)
print("→ geschrieben:", RUN_DIR / "wfcv_results_agg.csv", "und", RUN_DIR / "best_config.json")

Best config: {'feature_set': 'v2', 'features_used': 'mom+vol', 'n_features': 11, 'lookback': 60, 'width1': 32, 'width2': 16, 'dropout': 0.1, 'lr': 0.0005, 'cell': 'GRU', 'mcc_mean': 0.065751403238187, 'mcc_std': 0.03483550388528836, 'auprc_mean': 0.52533897424432, 'auprc_std': 0.032371146777913404, 'auroc_mean': 0.51403827184167, 'balacc_mean': 0.527664905383352, 'n_folds': 3}
→ geschrieben: ..\results\2025-10-22_12-13-21_wfcv\wfcv_results_agg.csv und ..\results\2025-10-22_12-13-21_wfcv\best_config.json


In [10]:
# ---- Score-Grids als Plot (robust) ------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt

agg_path = RUN_DIR / "wfcv_results_agg.csv"
assert agg_path.exists(), f"wfcv_results_agg.csv fehlt: {agg_path}"
agg = pd.read_csv(agg_path)   # <-- explizit laden (nicht auf 'g' verlassen)

# wähle sinnvolle Spalten-Kombinationen, die es wirklich gibt
pivot_index = "lookback" if "lookback" in agg.columns else None
col_candidates = ["features_used", "cell", "width1"]
pivot_columns = [c for c in col_candidates if c in agg.columns]
if not pivot_columns:  # Fallback, falls fast-Grid sehr klein ist
    pivot_columns = [agg.columns[0]]

(RUN_DIR / "plots").mkdir(parents=True, exist_ok=True)

def _plot_grid(df: pd.DataFrame, value_col: str, fname: str):
    if value_col not in df.columns:
        print(f"[WARN] Spalte {value_col} fehlt – skip Plot {fname}")
        return
    idx = pivot_index or pivot_columns[0]
    pvt = df.pivot_table(index=idx, columns=pivot_columns, values=value_col, aggfunc="mean")

    plt.figure(figsize=(10, 5))
    im = plt.imshow(pvt.values, aspect="auto")
    plt.colorbar(im)

    # Achsen-Beschriftungen
    plt.yticks(range(len(pvt.index)), [str(x) for x in pvt.index])
    if isinstance(pvt.columns, pd.MultiIndex):
        xt = [" | ".join(str(x) for x in tup) for tup in pvt.columns.to_list()]
    else:
        xt = [str(x) for x in pvt.columns.to_list()]
    plt.xticks(range(pvt.shape[1]), xt, rotation=45, ha="right")

    plt.title(fname.replace("_", " ").replace(".png", ""))
    plt.tight_layout()
    plt.savefig(RUN_DIR / "plots" / fname, dpi=160)
    plt.close()

# Heatmaps zeichnen (nur, wenn die Zielspalten vorhanden sind)
_plot_grid(agg, "mcc_mean",   "score_grid_mcc.png")
_plot_grid(agg, "auprc_mean", "score_grid_auprc.png")
print("✓ Plots geschrieben:", RUN_DIR / "plots" / "score_grid_mcc.png",
      " & ", RUN_DIR / "plots" / "score_grid_auprc.png")

✓ Plots geschrieben: ..\results\2025-10-22_12-13-21_wfcv\plots\score_grid_mcc.png  &  ..\results\2025-10-22_12-13-21_wfcv\plots\score_grid_auprc.png


In [11]:
# ---- Boxplots über Folds (MCC/AUPRC) -----------------------------------
# CHANGE: pro Konfiguration (kompakter Label) Boxplots der Fold-Verteilung speichern
def _short_label(r):
    return f"{r['features_used']}-{r['cell']}-{int(r['width1'])}/{int(r['width2'])}-lb{int(r['lookback'])}-dp{r['dropout']}-lr{r['lr']}"

if not results.empty:
    results["config_label"] = results.apply(_short_label, axis=1)

    # MCC
    plt.figure(figsize=(max(8, 0.35*len(results["config_label"].unique())), 5))
    data = [grp["mcc"].values for _, grp in results.groupby("config_label")]
    labels = list(results.groupby("config_label").groups.keys())
    plt.boxplot(data, showmeans=True, meanline=True)
    plt.xticks(range(1, len(labels)+1), labels, rotation=90)
    plt.title("MCC über Folds (pro Konfiguration)")
    plt.tight_layout()
    plt.savefig(RUN_DIR / "plots" / "boxplots_mcc.png", dpi=160)
    plt.close()

    # AUPRC
    plt.figure(figsize=(max(8, 0.35*len(results["config_label"].unique())), 5))
    data = [grp["auprc"].values for _, grp in results.groupby("config_label")]
    labels = list(results.groupby("config_label").groups.keys())
    plt.boxplot(data, showmeans=True, meanline=True)
    plt.xticks(range(1, len(labels)+1), labels, rotation=90)
    plt.title("AUPRC über Folds (pro Konfiguration)")
    plt.tight_layout()
    plt.savefig(RUN_DIR / "plots" / "boxplots_auprc.png", dpi=160)
    plt.close()

In [12]:
# ---- Run-Info dump ------------------------------------------------------
run_info = {
    "seed": SEED,
    "epochs_grid": EPOCHS_GRID,
    "n_folds": N_FOLDS,
    "val_frac": 0.20,
    "min_train_frac": 0.45,
    "lookback_grid": LOOKBACK_GRID,
    "hp_grid_size": (len(HP_GRID) if not FAST else 1),
    "feature_subsets": list(FEATURE_SUBSETS.keys()),
    "train_csv": TRAIN_CSV,
    "label_resolution": {
        "source": "yaml" if os.path.exists(yaml_path) and (label_h is not None) else "inferred_from_csv",
        "yaml_path": yaml_path
    },
    "labels": {"horizon": H_FOR_FILE, "mode": MODE_FOR_FILE, "epsilon": EPS_FOR_FILE},
    "notes": {
        "metric_mcc": "Validation MCC at best threshold per fold (not fixed 0.5)",  # CHANGE
        "boxplots": ["plots/boxplots_mcc.png", "plots/boxplots_auprc.png"]          # CHANGE
    }
}
with open(RUN_DIR / "wfcv_run_info.json", "w") as f:
    json.dump(run_info, f, indent=2)

print("\nBlock 5 abgeschlossen. Artefakte:")
print(" -", RUN_DIR / "wfcv_results.csv")
print(" -", RUN_DIR / "wfcv_results_agg.csv")
print(" -", RUN_DIR / "best_config.json")
print(" -", RUN_DIR / "wfcv_results_top5.csv")
print(" -", RUN_DIR / "plots")


Block 5 abgeschlossen. Artefakte:
 - ..\results\2025-10-22_12-13-21_wfcv\wfcv_results.csv
 - ..\results\2025-10-22_12-13-21_wfcv\wfcv_results_agg.csv
 - ..\results\2025-10-22_12-13-21_wfcv\best_config.json
 - ..\results\2025-10-22_12-13-21_wfcv\wfcv_results_top5.csv
 - ..\results\2025-10-22_12-13-21_wfcv\plots
